
# Opening a Distillery in the Finger Lakes Region of New York 

 ## Analysis for IBM Data Science Cert Capstone Project 


___


## Introduction: Business Problem

An alcoholic beverage distiller wants to open a new distillery in the Finger Lakes region of New York. The Finger Lakes region of Western New York is a very popular destination for tourism - especially for visiting vineyards, breweries and natural geographic features, such as gorges and waterfalls. The Finger Lakes region is very large and mostly rural. The distiller wants to find a location that will be close to similar businesses like wineries, and breweries, but also be located in an area that already gets a good amount of tourism activity.

The data science methodology tools will be used to identify and rank potential locations within the Finger Lakes region that are close to the most popular attractions while also being close to vineyards, wineries and breweries.

## Data

In order to identify the most desirable areas, based upon the distiller's preferences, we will need to analyze: 
 - the location of vineyards, wineries, breweries, and tourist attractions
 - the density of preferred venue types within each area
 - the popularity ranking of venues within each area, to be derived from Foursquare user activity

The Foursquare API will be used to provide venue information, location data, and venue popularity data. To support this analysis we will need the following data from the Foursquare API:

    - name
    - categories
    - latitude, longitude
    - number of check-ins
    - number of tips
    - trending data 
